In [1]:
!pip install layer-sdk -qqq

     |████████████████████████████████| 27.6 MB 62.7 MB/s 
     |████████████████████████████████| 1.3 MB 36.2 MB/s 
     |████████████████████████████████| 212 kB 59.9 MB/s 
     |████████████████████████████████| 40 kB 5.5 MB/s 
     |████████████████████████████████| 2.4 MB 59.6 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 15.5 MB 50.0 MB/s 
     |████████████████████████████████| 596 kB 64.1 MB/s 
     |████████████████████████████████| 271 kB 86.9 MB/s 
     |████████████████████████████████| 97 kB 7.9 MB/s 
     |████████████████████████████████| 159 kB 82.7 MB/s 
     |████████████████████████████████| 256 kB 52.6 MB/s 
     |████████████████████████████████| 3.6 MB 71.6 MB/s 
     |████████████████████████████████| 25.6 MB 1.4 MB/s 
     |████████████████████████████████| 3.1 MB 77.7 MB/s 
     |████████████████████████████████| 212.4 MB 71 kB/s 
     |████████████████████████████████| 374 kB 71.5 MB/s 
     |████████████

In [2]:
import layer
layer.login()

Please open the following link in your web browser. Once logged in, copy the code and paste it here.
https://auth.beta.layer.co/authorize?response_type=code&code_challenge=8xZnuC2ssMveE_BKwRmMVQyULmww3JUCzqX0Tnl9-Wk&code_challenge_method=S256&client_id=tvp1Wo8acTt4z46c7MbUB3Rg5MqfmqkN&redirect_uri=https://beta.layer.co/oauth/code&scope=offline_access&audience=https://beta.layer.co
Code: -XFDSaUX-ZCpXDJx
Successfully logged into https://beta.layer.co


In [3]:
data = layer.get_dataset("movie_ratings").to_spark()
data.show()

+------+-------+------+-------------------+
|USERID|MOVIEID|RATING|          TIMESTAMP|
+------+-------+------+-------------------+
| 35001|   3074|   4.0|2001-06-30 04:10:07|
| 35001|   3081|   3.0|2001-06-30 04:31:03|
| 35001|   3175|   3.0|2001-06-30 04:52:59|
| 35001|   3176|   2.0|2001-06-30 04:39:47|
| 35001|   3196|   3.0|2001-06-30 05:03:37|
| 35001|   3262|   2.0|2001-06-30 04:41:23|
| 35001|   3354|   2.0|2001-06-30 05:01:15|
| 35001|   3364|   2.0|2001-06-30 04:14:13|
| 35001|   3365|   4.0|2001-06-30 04:08:54|
| 35001|   3386|   3.0|2001-06-30 04:39:47|
| 35001|   3435|   4.0|2001-06-30 04:13:21|
| 35001|   3467|   3.0|2001-06-30 04:08:54|
| 35001|   3471|   3.0|2001-06-30 04:52:59|
| 35001|   3476|   4.0|2001-06-30 04:28:25|
| 35001|   3508|   3.0|2001-06-30 04:10:07|
| 35001|   3527|   4.0|2001-06-30 04:55:41|
| 35001|   3535|   3.0|2001-06-30 04:31:54|
| 35001|   3545|   5.0|2001-06-30 04:33:49|
| 35001|   3576|   5.0|2001-06-30 04:31:03|
| 35001|   3671|   2.0|2001-06-3

In [4]:
# Split the data into a training and testing set
training_size = 0.8
random_state = 0
test_size = 0.2
training, testing = data.randomSplit([training_size, test_size], seed=random_state)

In [5]:
test_user = testing.filter(testing['USERID']==23).select(['MOVIEID','USERID'])

In [6]:
spark_recommender_system = layer.get_model("spark_recommender_system")

In [7]:
print(spark_recommender_system.parameters)
print(spark_recommender_system.metrics)

{'training_size': '0.7', 'test_size': '0.3', 'coldStartStrategy': 'drop', 'maxIter': '5', 'ratingCol': 'RATING', 'regParam': '0.01', 'random_state': '0', 'userCol': 'USERID', 'itemCol': 'MOVIEID'}
{'RMSE': [(1638426879434, 0.8191235838006905)]}


In [8]:
spark_recommender = spark_recommender_system.get_train()

In [9]:
test_user_recommendations = spark_recommender.transform(test_user)


In [10]:
test_user_recommendations.orderBy('prediction',ascending=False).show(5)

+-------+------+----------+
|MOVIEID|USERID|prediction|
+-------+------+----------+
|    293|    23|   4.84422|
|    318|    23| 4.7918367|
|   2324|    23| 4.4672356|
|   2278|    23|  4.381276|
|   1527|    23|   4.37734|
+-------+------+----------+
only showing top 5 rows



In [11]:
movies = layer.get_dataset("movie").to_spark()
movies.show()

+-------+--------------------+--------------------+
|MOVIEID|               TITLE|              GENRES|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [12]:
recommended_movies = test_user_recommendations.join(movies, ['MOVIEID'])

In [14]:
recommended_movies.show(5)

+-------+------+----------+--------------------+--------------------+
|MOVIEID|USERID|prediction|               TITLE|              GENRES|
+-------+------+----------+--------------------+--------------------+
|      1|    23| 3.7771487|    Toy Story (1995)|Adventure|Animati...|
|    293|    23|   4.84422|Léon: The Profess...|Action|Crime|Dram...|
|    318|    23| 4.7918367|Shawshank Redempt...|         Crime|Drama|
|    356|    23| 4.1593113| Forrest Gump (1994)|Comedy|Drama|Roma...|
|    493|    23| 4.2100744|Menace II Society...|  Action|Crime|Drama|
+-------+------+----------+--------------------+--------------------+
only showing top 5 rows

